In [118]:
# magic command to display matplotlib plots inline within the ipython notebook webpage
%matplotlib inline

# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import matplotlib.cm as cm, matplotlib.font_manager as fm
import datetime
from scipy import stats, integrate

In [119]:
herepath = pd.read_csv('data/MOW_Path_Speed_5Min_2014_DOW_SpeedLimit.csv', encoding='utf-8')
herepath.head()

,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,SPDLIMIT,MEAN,STDDEV,MIN,MAX,CONFIDENCE,PCT_5,PCT_50,PCT_85,DOW
0,998969850F,2014-01-01 13:55,167,186,72,61.0,0.0,61,61,10,61,61,61,Wednesday
1,998969850F,2014-01-02 11:05,133,186,72,60.0,0.0,60,60,10,60,60,60,Thursday
2,998969850F,2014-01-02 13:20,160,186,72,61.0,0.0,61,61,10,61,61,61,Thursday
3,998969850F,2014-01-02 14:30,174,186,72,52.0,0.0,52,52,10,52,52,52,Thursday
4,998969850F,2014-01-03 09:05,109,186,72,57.0,0.0,57,57,10,57,57,57,Friday


In [120]:
#For now, enter the weekdays desire here in full
Weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
Weekday

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

In [121]:
Weekend = ['Saturday','Sunday']
Weekend

['Saturday', 'Sunday']

In [122]:
Both = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
Both

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [123]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe2(x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        days = v[v['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [124]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsALL(x):
    y = pd.DataFrame(columns=[])
    
    y['MEAN_ALL'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_85thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_5thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thAll'] = epoct
    y['PcntInterval_85thAll'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thAll'] = (np.sqrt(Var))*(0.6213)
    
    y = np.round(y,3)
    return y

In [125]:
def IndecesALLAM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_SL_AM'] = y['SPDLIMIT'].div(y['MEAN_ALL'])
    z['PTI_SL_AM'] = y['SPDLIMIT'].div(y['PcntSpeed_5thAll'])
    z['TTI_85thALL_AM'] = y['PcntSpeed_85thAll'].div(y['MEAN_ALL'])
    z['PTI_85thALL_AM'] = y['PcntSpeed_85thAll'].div(y['PcntSpeed_5thAll'])
    z['BTI_AM'] = (y['MEAN_ALL'].div(y['PcntSpeed_5thAll']))-1
    z = np.round(z,3)
    return z

In [126]:
def IndecesALLMD(y):
    z = pd.DataFrame(columns = [])
    z['TTI_SL_MD'] = y['SPDLIMIT'].div(y['MEAN_ALL'])
    z['PTI_SL_MD'] = y['SPDLIMIT'].div(y['PcntSpeed_5thAll'])
    z['TTI_85th_MD'] = y['PcntSpeed_85thAll'].div(y['MEAN_ALL'])
    z['PTI_85th_MD'] = y['PcntSpeed_85thAll'].div(y['PcntSpeed_5thAll'])
    z['BTI_MD'] = (y['MEAN_ALL'].div(y['PcntSpeed_5thAll']))-1
    z = np.round(z,3)
    return z

In [127]:
def IndecesALLPM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_SL_PM'] = y['SPDLIMIT'].div(y['MEAN_ALL'])
    z['PTI_SL_PM'] = y['SPDLIMIT'].div(y['PcntSpeed_5thAll'])
    z['TTI_85th_PM'] = y['PcntSpeed_85thAll'].div(y['MEAN_ALL'])
    z['PTI_85th_PM'] = y['PcntSpeed_85thAll'].div(y['PcntSpeed_5thAll'])
    z['BTI_PM'] = (y['MEAN_ALL'].div(y['PcntSpeed_5thAll']))-1
    z = np.round(z,3)
    return z

In [128]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWd(x):
    y = pd.DataFrame(columns=[])
    y['MEAN_Wkd'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)

    
    y['PcntSpeed_85thWd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_5thWd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thWd'] = epoct
    y['PcntInterval_85thWd'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thWd'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [129]:
def IndecesWdAM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_85thWd_AM'] = y['PcntSpeed_85thWd'].div(y['MEAN_Wkd'])
    z['PTI_85thWd_AM'] = y['PcntSpeed_85thWd'].div(y['PcntSpeed_5thWd'])
    z = np.round(z,3)
    return z

In [130]:
def IndecesWdMD(y):
    z = pd.DataFrame(columns = [])
    z['TTI_85thWd_MD'] = y['PcntSpeed_85thWd'].div(y['MEAN_Wkd'])
    z['PTI_85thWd_MD'] = y['PcntSpeed_85thWd'].div(y['PcntSpeed_5thWd'])
    z = np.round(z,3)
    return z

In [131]:
def IndecesWdPM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_85thWd_PM'] = y['PcntSpeed_85thWd'].div(y['MEAN_Wkd'])
    z['PTI_85thWd_PM'] = y['PcntSpeed_85thWd'].div(y['PcntSpeed_5thWd'])
    z = np.round(z,3)
    return z

In [132]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWend(x):
    y = pd.DataFrame(columns=[])
    y['MEAN_WEnd'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_85thWend'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thWend'] = epoct
    y['PcntInterval_85thWend'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thWend'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [133]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdtime(x):
    y = pd.DataFrame(columns=[])
    y['MEAN_Wdtime'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_60thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_85thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_5thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_Wdtime'] = epoct
    y['PcntInterval_Wdtime'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_Wdtime'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [134]:
def IndecesWDtimeAM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_60thWdtime_AM'] = y['PcntSpeed_60thWdtime'].div(y['MEAN_Wdtime'])
    z['PTI_60thWdtime_AM'] = y['PcntSpeed_60thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z['TTI_85thWdtime_AM'] = y['PcntSpeed_85thWdtime'].div(y['MEAN_Wdtime'])
    z['PTI_85thWdtime_AM'] = y['PcntSpeed_85thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z = np.round(z,3)
    return z

In [135]:
def IndecesWDtimeMD(y):
    z = pd.DataFrame(columns = [])
    z['TTI_60thWdtime_MD'] = y['PcntSpeed_60thWdtime'].div(y['MEAN_Wdtime'])
    z['PTI_60thWdtime_MD'] = y['PcntSpeed_60thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z['TTI_85thWdtime_MD'] = y['PcntSpeed_85thWdtime'].div(y['MEAN_Wdtime'])
    z['PTI_85thWdtime_MD'] = y['PcntSpeed_85thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z = np.round(z,3)
    return z

In [136]:
def IndecesWDtimePM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_60thWdtime_PM'] = y['PcntSpeed_60thWdtime'].div(y['MEAN_Wdtime'])
    z['PTI_60thWdtime_PM'] = y['PcntSpeed_60thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z['TTI_85thWdtime_PM'] = y['PcntSpeed_85thWdtime'].div(y['MEAN_Wdtime'])
    z['PTI_85thWdtime_PM'] = y['PcntSpeed_85thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z = np.round(z,3)
    return z

In [137]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayAM(x):
    y = pd.DataFrame(columns=[])
    y['MEAN_WkdAM'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayAM'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayAM'] = y['MEAN_WkdAM']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayAM'] = epoct
    y['PcntInterval_WdayAM'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayAM'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [138]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayMD(x):
    y = pd.DataFrame(columns=[])
    y['MEAN_WkdMD'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayMD'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayMD'] = y['MEAN_WkdMD']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayMD'] = epoct
    y['PcntInterval_WdayMD'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayMD'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [139]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayPM(x):
    y = pd.DataFrame(columns=[])
    y['MEAN_WkdPM'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayPM'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayPM'] = y['MEAN_WkdPM']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayPM'] = epoct
    y['PcntInterval_WdayPM'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayPM'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [140]:
#For all days and all time periods
All = timeframe2(herepath,0,288,False,Both)
bylink = HEREstatsALL(All)
bylink = bylink.drop(['SPDLIMIT'], axis = 1)
All = pd.DataFrame()
bylink.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll
LINK_DIR,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.444
107030439T,35.148,44.734,17.396,7857,7.474,9.929
107030440T,34.863,45.355,17.086,7325,6.968,10.303
107030444F,36.180,47.219,16.154,9311,8.857,11.209
107030445F,36.602,47.840,16.775,9256,8.805,11.250


In [141]:
#for weekdays all links
Wkd = timeframe2(herepath,0,288,True,Weekday)
SpeedWd = HEREstatsWd(Wkd)
SpeedWd = SpeedWd.drop(['SPDLIMIT'], axis = 1)
Wkd = pd.DataFrame()
SpeedWd.head()

MemoryError: 

In [ ]:
#for weekends all links
Wend = timeframe2(herepath,0,288,True,Weekend)
SpeedWend = HEREstatsWend(Wend)
SpeedWend = SpeedWend.drop(['SPDLIMIT'], axis = 1)
Wend = pd.DataFrame()
SpeedWend.head()

In [ ]:
#Weekday Between 6AM and 8PM all links
Wdtime = timeframe2(herepath,72,240,True,Weekday)
SpeedWdtime = HEREstatsWdtime(Wdtime)
SpeedWdtime = SpeedWdtime.drop(['SPDLIMIT'], axis = 1)
Wdtime = pd.DataFrame()
SpeedWdtime.head()

In [ ]:
#Stats between 6AM and 9AM all links
WdtimeAM = timeframe2(herepath,72,108,True,Weekday)
SpeedWdayAM = HEREstatsWdayAM(WdtimeAM)
SpeedWdayAM = SpeedWdayAM.drop(['SPDLIMIT'], axis = 1)
WdtimeAM = pd.DataFrame()
SpeedWdayAM.head()

In [ ]:
#Stats between 9AM and 3PM all links
WdtimeMD = timeframe2(herepath,108,180,True,Weekday)
SpeedWdayMD = HEREstatsWdayMD(WdtimeMD)
SpeedWdayMD = SpeedWdayMD.drop(['SPDLIMIT'], axis = 1)
WdtimeMD = pd.DataFrame()
SpeedWdayMD.head()

In [ ]:
#Stats between 3PM and 6PM all links
WdtimePM = timeframe2(herepath,180,216,True,Weekday)
SpeedWdayPM = HEREstatsWdayPM(WdtimePM)
SpeedWdayPM = SpeedWdayPM.drop(['SPDLIMIT'], axis = 1)
WdtimePM = pd.DataFrame()
SpeedWdayPM.head()

In [ ]:
#Calculates AM Indeces for all 365 days for all links
AllAM = timeframe2(herepath,72,108,False,Both)
HstatsAM = HEREstatsALL(AllAM)
IndecesAllAM = IndecesALLAM(HstatsAM)
AllAM = pd.DataFrame()
IndecesAllAM.head()

In [ ]:
#Calculates MD Indeces for all 365 days for all links
AllMD = timeframe2(herepath,108,180,False,Both)
HstatsMD = HEREstatsALL(AllMD)
IndecesAllMD = IndecesALLMD(HstatsMD)
AllMD = pd.DataFrame()
IndecesAllMD.head()

In [ ]:
#Calculates PM Indeces for all 365 days for all links
AllPM = timeframe2(herepath,180,216,False,Both)
HstatsPM = HEREstatsALL(AllPM)
IndecesAllPM = IndecesALLPM(HstatsPM)
AllPM = pd.DataFrame()
IndecesAllPM.head()

In [ ]:
#Calculates weekday non-holiday indeces for AM for all links
WdAM = timeframe2(herepath,72,108,True,Weekday)
HstatsWdAM = HEREstatsWd(WdAM)
IndecesWdayAM = IndecesWdAM(HstatsWdAM)
WdAM = pd.DataFrame()
IndecesWdayAM.head()

In [ ]:
#Calculates weekday non-holiday indeces for MD for all links
WdMD = timeframe2(herepath,108,180,True,Weekday)
HstatsWdMD = HEREstatsWd(WdMD)
IndecesWdayMD = IndecesWdMD(HstatsWdMD)
WdMD = pd.DataFrame()
IndecesWdayMD.head()

In [ ]:
#Calculates weekday non-holiday indeces for PM for all links
WdPM = timeframe2(herepath,180,216,True,Weekday)
HstatsWdPM = HEREstatsWd(WdPM)
IndecesWdayPM = IndecesWdPM(HstatsWdPM)
WdPM = pd.DataFrame()
IndecesWdayPM.head()

In [ ]:
#Calculates daytime indeces for AM for all links
WdtimeAM = timeframe2(herepath,72,108,True,Weekday)
HstatsWdtimeAM = HEREstatsWdtime(WdtimeAM)
IndecesWdtimeAM = IndecesWDtimeAM(HstatsWdtimeAM)
WdtimeAM = pd.DataFrame()
IndecesWdtimeAM.head()

In [ ]:
#Calculates daytime indeces for MD for all links
WdtimeMD = timeframe2(herepath,108,180,True,Weekday)
HstatsWdtimeMD = HEREstatsWdtime(WdtimeMD)
IndecesWdtimeMD = IndecesWDtimeMD(HstatsWdtimeMD)
WdtimeMD = pd.DataFrame()
IndecesWdtimeMD.head()

In [ ]:
#Calculates daytime indeces for PM for all links
WdtimePM = timeframe2(herepath,180,216,True,Weekday)
HstatsWdtimePM = HEREstatsWdtime(WdtimePM)
IndecesWdtimePM = IndecesWDtimePM(HstatsWdtimePM)
WdtimePM = pd.DataFrame()
IndecesWdtimePM.head()

In [ ]:
master = bylink

In [ ]:
master = pd.merge(master, SpeedWd, left_index = True, right_index = True)
SpeedWd = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, SpeedWend, left_index = True, right_index = True)
SpeedWend = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, SpeedWdtime, left_index = True, right_index = True)
SpeedWdtime = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, SpeedWdayAM, left_index = True, right_index = True)
SpeedWdayAM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, SpeedWdayMD, left_index = True, right_index = True)
SpeedWdayMD = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, SpeedWdayPM, left_index = True, right_index = True)
SpeedWdayPM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesAllAM, left_index = True, right_index = True)
IndecesAllAM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesAllMD, left_index = True, right_index = True)
IndecesAllMD = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesAllPM, left_index = True, right_index = True)
IndecesAllPM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesWdayAM, left_index = True, right_index = True)
IndecesWdayAM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesWdayMD, left_index = True, right_index = True)
IndecesWdayMD = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesWdayPM, left_index = True, right_index = True)
IndecesWdayPM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesWdtimeAM, left_index = True, right_index = True)
IndecesWdtimeAM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesWdtimeMD, left_index = True, right_index = True)
IndecesWdtimeMD = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesWdtimePM, left_index = True, right_index = True)
IndecesWdtimePM = pd.DataFrame()
master

In [ ]:
master.to_csv('HEREstatsWMEAN.csv')